<a href="https://colab.research.google.com/github/Lakith-Rambukkanage/SigmaLaw-WPP/blob/main/sentence_embeddings/Sentence_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Drive Mounting

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# sample_data_path = '/content/drive/Shareddrives/SigmaLaw-WPP/LRR/sample_preprocessed'


In [ ]:
!cp /content/drive/Shareddrives/SigmaLaw-WPP/LRR/sentences_and_decision.csv "/content/"

In [ ]:
import pandas as pd
df = pd.read_csv("sentences_and_decision.csv")
df_all = pd.read_csv("sentences_and_decision.csv",converters={"sentences": literal_eval})
df_all.drop(columns=[df_all.columns[0],df_all.columns[1]],inplace=True)
df_all.head()

,sentences,Y
0,['mr george s wright of dallas tex for respond...,0
1,['prior to this decision however owing to doub...,0
2,['mr justice roberts delivered the opinion of ...,1
3,['b a different result is not required by the ...,0
4,['appellees prosecution for burning a flag in ...,0


In [ ]:
def dropSent(sentences):
  if(len(sentences)>=150):
    sentences = sentences[:150]
  return sentences
df_all["sentences"] = df_all["sentences"].apply(dropSent)
len(df_all.iloc[0]["sentences"])

150

In [ ]:
df = df_all.loc[0:10,["sentences","Y"]]
df.head()

## Package installs and imports

In [ ]:
## Install libraries
! python -m pip install --upgrade pip --quiet
! pip install spacy --quiet
! pip3 install spacy-transformers --quiet
! python -m spacy download en_core_web_trf --quiet

In [ ]:
import os
import spacy



# Test data

In [ ]:
test_case_list = {'case1': ['post,at emphasis added',
 'but we see no call to suppose that congress regarded state prosecutions as grapefruit league versions of the big show'],
 'case2': ['mistretta united states,scalia,dissenting',
 'in our federal system,states possess primary authority for defining and enforcing criminal laws,including those prohibiting the gravest crimes'],
 'case3': ['for that reason,even when attorneys have jurisdiction,they are generally to defer to,rather than supplant,state prosecutions of serious offenses',
 'see attorneys manual principles of federal prosecution',
 'and still more obviously,the dissent theory fails with respect to foreign convictions']}

test_df = pd.DataFrame([test_case_list]).transpose()
test_df

,0
case1,"[post,at emphasis added, but we see no call to..."
case2,"[mistretta united states,scalia,dissenting, in..."
case3,"[for that reason,even when attorneys have juri..."


# Load RoBERTa

In [ ]:
roberta = spacy.load('en_core_web_trf')


In [ ]:
test_embed = []
for doc in roberta.pipe(['to to to']):
    test_embed.append(doc._.trf_data.tensors[-1])

test_embed

[array([[ 0.27845278,  0.09190975, -0.42502975,  0.194725  ,  0.08745601,
          0.05117087,  0.46064234,  0.08891254, -0.58169806, -0.3087634 ,
          0.49685693, -0.4873821 , -0.01159835,  0.6829977 , -0.4201913 ,
          0.33239883, -0.0011053 , -0.6898292 , -0.06298554,  0.24491104,
          0.095768  , -0.06418903,  0.20491098,  0.23142546,  0.3249031 ,
          0.29678833,  0.08238516, -0.46309158,  0.5701739 ,  0.53220284,
          0.27187228, -0.3712688 , -0.08509661,  0.49091217, -0.35999417,
          0.48415062, -0.41453987,  0.37985197,  0.3902132 , -0.4209872 ,
         -0.30571195,  0.12397907, -0.04832345,  0.21610478,  0.17758353,
          0.33379477,  0.07160388,  0.13556452, -0.3908229 ,  0.33462176,
         -0.28077435,  0.5045948 ,  0.2590427 ,  0.41506076, -0.14329784,
          0.29474238,  0.21214883,  0.20789847, -0.3003495 , -0.05670163,
         -0.50618184, -0.28960943,  0.37449878, -0.0733954 , -0.20779933,
         -0.15620282, -0.39394718, -0.

In [ ]:
def encodeRoBERTa(sentence_list):
  sentence_embedding_list = []
  for doc in roberta.pipe(sentence_list):
    sentence_embedding_list.append(doc._.trf_data.tensors[-1])
  return sentence_embedding_list

In [ ]:
test_df['RoBERTa']=test_df[0].apply(encodeRoBERTa)

In [ ]:
# test_df.iloc[0]['embedding']
test_df

,0,RoBERTa
case1,"[post,at emphasis added, but we see no call to...","[[[0.35047665, 0.027662192, -0.22531478, -0.06..."
case2,"[mistretta united states,scalia,dissenting, in...","[[[0.20373441, -0.2814409, -0.37168983, -0.027..."
case3,"[for that reason,even when attorneys have juri...","[[[0.2921267, -0.018840646, -0.20124677, 0.049..."


In [ ]:
len(df.iloc[0]['RoBERTa'][0][0])

768

## Sentence BERT

In [ ]:
!pip install spacy_sentence_bert --quiet

     |████████████████████████████████| 81kB 9.5MB/s 
     |████████████████████████████████| 12.8MB 247kB/s 
     |████████████████████████████████| 2.5MB 49.2MB/s 
     |████████████████████████████████| 1.2MB 49.3MB/s 
     |████████████████████████████████| 624kB 48.3MB/s 
     |████████████████████████████████| 51kB 9.1MB/s 
     |████████████████████████████████| 460kB 54.4MB/s 
     |████████████████████████████████| 9.1MB 154kB/s 
     |████████████████████████████████| 3.3MB 34.1MB/s 
     |████████████████████████████████| 901kB 33.5MB/s 
     |████████████████████████████████| 122kB 52.0MB/s 


In [ ]:

from sentence_transformers import SentenceTransformer
sbert = SentenceTransformer('stsb-roberta-base-v2')

In [ ]:
def encode(sentences):
  embedding = [sbert.encode(sent) for sent in sentences]#,show_progress_bar=True)]
  return embedding
len(encode(["hi im some","yeah okay"]))

2

In [ ]:
df['SBERT'] = df["sentences"].apply(encode)

In [ ]:
df.head()

,sentences,Y,SBERT
0,['mr george s wright of dallas tex for respond...,0,"[-0.20961645, -0.23212002, 0.89044833, 0.27971..."
1,['prior to this decision however owing to doub...,0,"[0.58679605, 0.6073739, 0.06551086, -0.0438442..."
2,['mr justice roberts delivered the opinion of ...,1,"[-0.048205514, -0.0796765, 0.40064067, -0.4417..."
3,['b a different result is not required by the ...,0,"[0.06357362, 1.1819575, 0.4121081, -0.26700753..."
4,['appellees prosecution for burning a flag in ...,0,"[-0.5459659, -0.25448644, 0.4573704, -0.740932..."


In [ ]:
len(df.iloc[0]['SBERT'][0])

TypeError: ignored

## Universal Sentence Encoder

In [ ]:
!pip install spacy-universal-sentence-encoder --quiet

In [ ]:
import spacy_universal_sentence_encoder
use_model = spacy_universal_sentence_encoder.load_model('en_use_lg')

In [ ]:
def encodeUSE(sentence_list):
  sentence_embedding_list = []
  for doc in use_model.pipe(sentence_list):
    sentence_embedding_list.append(doc.vector)
  return sentence_embedding_list

In [ ]:
test_df['USE']=test_df[0].apply(encodeUSE)

In [ ]:
len(test_df.iloc[0]['USE'][0])

512

In [ ]:
import tensorflow as tf

tk = tf.keras.preprocessing.text.Tokenizer()
tk.fit_on_texts(sample_df['text'])

tf_idf_samples = tk.sequences_to_matrix(tk.texts_to_sequences(sample_df['text']), mode='tfidf')
tf_idf_samples


array([[0.        , 5.15290727, 4.77430413, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 4.16781383, 3.6020304 , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 4.30271698, 3.80744861, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 5.37481813, 4.79399345, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 4.53581436, 4.21850062, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 5.01921731, 4.76147476, ..., 3.93182563, 3.93182563,
        3.93182563]])

# Saving

In [ ]:
# avoid risk
save_df = df.drop(columns=["sentences"])
save_df.to_csv("safe.csv")

In [ ]:
!cp safe.csv "/content/drive/Shareddrives/SigmaLaw-WPP/LRR/"

In [ ]:
flat_func = lambda x: np.array(x).flatten()
# df[key]=df[key].apply(flat_func)

In [ ]:
import tensorflow as tf
pad_arr = [0 for i in range(768)]#DIM
def pad(array_of_embed):
  return tf.keras.preprocessing.sequence.pad_sequences(
    array_of_embed, 
    maxlen=150, 
    # dtype="int32", 
    padding="post", 
    truncating="post", 
    value=pad_arr
    ).tolist()

# pad([[[1,2,3],[1,2,4]],[[1,3,4]]])
# test_df[key]= pad(test_df[key])

In [ ]:
# df[key]= pad(df[key])